# The Battle of Neighborhoods
### Charlotte Huang
### Oct 2019
This notebook is for the capstone project of the IBM Data Science Certificate. <br>
This project aims to solve the question: if you would like to move to another community for a job offer, how can you decide which community to relocate to? You need to compare communities, for example, with amenitieis, the other types of venues that exist in the neighborhood, such as gourmet fast food joints, pharmacies, parks, grad schools and so on. This project solves the problem by analyzing the location data

In [1]:
# import packages
import numpy as np
import pandas as pd

In [2]:
print "Hello Capstone Project Course!"

Hello Capstone Project Course!
